In [1]:
pip install langchain langchain_community langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [3]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

 ········


In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
# Load, chunk and index the contents of the blog.
loader = TextLoader("/Users/ynie/Downloads/OTL_edited.csv")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition refers to breaking down a complex problem into smaller, manageable tasks that can be solved individually. This method allows for better understanding and application of solutions, often leading to the discovery of patterns or methods that can be reused in similar problems. It is a fundamental aspect of problem-solving across various disciplines, including computing and mathematics.'

In [11]:
rag_chain.invoke("Who wrote College Algebra?")

'College Algebra was written by Carl Stitz and Jeff Zeager. There is also a version called College Algebra 2e authored by Jay Abramson.'

In [12]:
rag_chain.invoke("tell me about Launch! Advertising and Promotion in Real Time")

'"Launch! Advertising and Promotion in Real Time," authored by Michael Solomon, Lisa Duke Cornell, and Amit Nizan, is a textbook designed for advertising and promotion courses. It uniquely teaches advertising concepts by reverse engineering a real campaign, specifically SS+K\'s campaign for msnbc.com, providing insights into both agency and client perspectives. The book offers a detailed look at the advertising process, enhancing students\' understanding of practical applications of theory in the field.'